# Домашнее задание № 3. Исправление опечаток

## 1. Учет грамматики при оценке исправлений (3 балла)

В последнюю итерацию алгоритма для генерации исправлений добавьте еще один компонент - учет грамматической информации. Частично она уже учитывается за счет языковой модели (вероятность предсказывается для словоформы), но такой подход ограничен из-за того, что модель не может ничего предсказать для словоформ, которых не было в обучающей выборке. Чтобы это исправить постройте еще одну "языковую модель" на грамматических тэгах:
1) Используя mystem или pymorphy, разметьте какой-нибудь корпус (например, кусок wiki из семинара) или воспользуйтесь уже размеченным корпусом (например, opencorpora)
2) соберите униграмные и биграмные статистики на уровне грамматических тэгов (например, вместо `задача важна` у вас будет биграм `S,жен,неод=им,ед A=ед,кр,жен`). Для простоты можете начать только с частеречных тэгов и добавить остальную информацию позже
3) напишите функцию, которая будет оценивать вероятность данного предложения на основе грамматической языковой модели (статистик из предыдущего шага). Функция должна сначала преобразовать текст в грамматические тэги, используя точно такой же подход, что использовался на шаге 1. 
4) в функции correct_text_with_lm замените compute_sentence_proba на вашу новую функцию и прогоните получившийся алгоритм на данных
5) сравните предсказания с предсказанием изначального correct_text_with_lm, проверьте метрики и посмотрите на различие в ошибках и исправлениях, найдите несколько примеров отличий в предсказаниях этих подходов

In [4]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
import itertools
from pprint import pprint
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm
import pymorphy3

In [5]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [6]:
substrings = tokenize(corpus)
tokens = [token.text for token in substrings]

In [7]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [8]:
vocab = Counter(re.findall('\w+', corpus.lower()))
N = sum(vocab.values())

<>:1: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
<>:1: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
C:\Users\MSI\AppData\Local\Temp\ipykernel_9772\2788637254.py:1: SyntaxWarning: "\w" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\w"? A raw string is also an option.
  vocab = Counter(re.findall('\w+', corpus.lower()))


In [9]:
def P(word, N=N):
    return vocab[word] / N

def known(words):
    return set(w for w in words if w in vocab)

def edits1(word):
    letters = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def correction(word):
    return max(candidates(word), key=P)

def calculate_metrics(true, actual, predicted):
    correct = 0
    total = 0

    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    for i in range(len(true)):
        t, a, p = true[i], actual[i], predicted[i]
    
        if t == p:
            correct += 1
        total += 1
        
        if t == a:
            total_correct += 1
            if t != p:
                correct_broken += 1

        else:
            total_mistaken += 1
            if t == p:
                mistaken_fixed += 1

    return {
        "total_accuracy": correct/total,
        "fixed_mistakes": mistaken_fixed/total_mistaken if total_mistaken > 0 else 0,
        "broken_correct_words": correct_broken/total_correct if total_correct > 0 else 0
    }

In [10]:
sentences = list(sentenize(corpus))
train_size = int(len(sentences) * 0.9)
train_sentences = sentences[:train_size]

In [11]:
morph = pymorphy3.MorphAnalyzer()

def get_word_tags(word):
    parsed = morph.parse(word)[0]
    return str(parsed.tag)

tagged_sentences = []
for sent in tqdm(train_sentences):
    
    tokens = [t.text for t in tokenize(sent.text)]

    tokens = [t.lower() for t in tokens if t.lower() not in punct and t.strip()]
    
    if not tokens:
        continue
    
    sent_tags = []
    for token in tokens:
        try:
            tag = get_word_tags(token)
            sent_tags.append((token, tag))
        except:
            continue
    
    if sent_tags:
        tagged_sentences.append(sent_tags)

print(f"Размечено предложений: {len(tagged_sentences)}")

  0%|          | 0/174519 [00:00<?, ?it/s]

Размечено предложений: 174488


In [12]:
unigram_tags = Counter()
bigram_tags = Counter()
start_tags = Counter()

for sent_tags in tqdm(tagged_sentences):
    if not sent_tags:
        continue
    
    tags = [tag for _, tag in sent_tags]
    
    for tag in tags:
        unigram_tags[tag] += 1
    
    for i in range(1, len(tags)):
        bigram = (tags[i-1], tags[i])
        bigram_tags[bigram] += 1
    
    start_tag = tags[0]
    start_tags[start_tag] += 1

  0%|          | 0/174488 [00:00<?, ?it/s]

In [19]:
class GrammarLanguageModel:
    
    def __init__(self, smoothing=0.1):

        self.smoothing = smoothing
        self.tag_counts = Counter()
        self.bigram_counts = Counter()
        self.start_counts = Counter()
        
        self.tag_probs = {}
        self.bigram_probs = {}
        self.start_probs = {}
        
        self.total_tags = 0
        self.total_starts = 0
        self.vocab_size = 0
        
        self.morph = pymorphy3.MorphAnalyzer()
        
    def get_grammar_tag(self, word):
        
        try:
            parsed = self.morph.parse(word)[0]
            tag_str = str(parsed.tag)
            
            return tag_str
        except Exception as e:
            return 'UNK'
    
    def train(self, sentences):
        self.tag_counts.clear()
        self.bigram_counts.clear()
        self.start_counts.clear()
        
        self.total_tags = 0
        self.total_starts = 0
        
        for i, sentence in enumerate(tqdm(sentences)):
            tokens = [t.text.lower() for t in tokenize(sentence)]
            
            tokens = [t for t in tokens if t not in punct and t.strip()]
            
            if not tokens:
                continue
            
            tags = []
            for token in tokens:
                tag = self.get_grammar_tag(token)
                tags.append(tag)
            
            first_tag = tags[0]
            self.start_counts[first_tag] += 1
            self.total_starts += 1
            
            for tag in tags:
                self.tag_counts[tag] += 1
                self.total_tags += 1
            
            for j in range(1, len(tags)):
                prev_tag = tags[j-1]
                curr_tag = tags[j]
                self.bigram_counts[(prev_tag, curr_tag)] += 1
        
        self._compute_probabilities()
    
    def _compute_probabilities(self):
        V = len(self.tag_counts)
        
        self.tag_probs = {}
        
        for tag, count in self.tag_counts.items():
            numerator = count + self.smoothing
            
            denominator = self.total_tags + self.smoothing * V
            
            prob = numerator / denominator
            self.tag_probs[tag] = prob
        
        self.unk_prob = self.smoothing / (self.total_tags + self.smoothing * V)
        
        self.bigram_probs = {}
        
        for (tag1, tag2), count in self.bigram_counts.items():
            tag1_count = self.tag_counts.get(tag1, 0)
            
            numerator = count + self.smoothing
            
            denominator = tag1_count + self.smoothing * V
            
            if denominator > 0:
                prob = numerator / denominator
                self.bigram_probs[(tag1, tag2)] = prob
        
        self.start_probs = {}
        if self.total_starts > 0:
            for tag, count in self.start_counts.items():
                self.start_probs[tag] = count / self.total_starts
        
        self.vocab_size = V
    
    def get_tag_probability(self, tag):
        return self.tag_probs.get(tag, self.unk_prob)
    
    def get_transition_probability(self, tag1, tag2):
        prob = self.bigram_probs.get((tag1, tag2))
        
        if prob is not None:
            return prob
        
        tag1_count = self.tag_counts.get(tag1, 0)
        
        numerator = self.smoothing
        denominator = tag1_count + self.smoothing * self.vocab_size
        
        if denominator > 0:
            return numerator / denominator
        else:
            return self.unk_prob
    
    def get_start_probability(self, tag):
        return self.start_probs.get(tag, 0.0)
    
    def compute_sentence_probability(self, sentence_text, use_log=True):
        tokens = [t.text.lower() for t in tokenize(sentence_text)]
        tokens = [t for t in tokens if t not in punct and t.strip()]
        
        if not tokens:
            return -np.inf if use_log else 0.0
        
        tags = []
        for token in tokens:
            tag = self.get_grammar_tag(token)
            tags.append(tag)
        
        if use_log:
            log_prob = 0.0
            
            first_tag = tags[0]
            start_prob = self.get_start_probability(first_tag)
            
            if start_prob > 0:
                log_prob += np.log(start_prob)
            else:
                tag_prob = self.get_tag_probability(first_tag)
                log_prob += np.log(tag_prob)
            
            for i in range(1, len(tags)):
                tag1 = tags[i-1]
                tag2 = tags[i]
                
                trans_prob = self.get_transition_probability(tag1, tag2)
                log_prob += np.log(trans_prob)
            
            return log_prob
            
        else:
            prob = 1.0
            
            first_tag = tags[0]
            start_prob = self.get_start_probability(first_tag)
            
            if start_prob > 0:
                prob *= start_prob
            else:
                prob *= self.get_tag_probability(first_tag)
            
            for i in range(1, len(tags)):
                tag1 = tags[i-1]
                tag2 = tags[i]
                
                trans_prob = self.get_transition_probability(tag1, tag2)
                prob *= trans_prob
            
            return prob
    
    def compare_sentences(self, sentence1, sentence2):
        prob1 = self.compute_sentence_probability(sentence1, use_log=True)
        prob2 = self.compute_sentence_probability(sentence2, use_log=True)
        
        return {
            'sentence1': sentence1,
            'sentence2': sentence2,
            'log_prob1': prob1,
            'log_prob2': prob2,
            'more_probable': 1 if prob1 > prob2 else 2
        }

grammar_lm = GrammarLanguageModel(smoothing=0.1)
grammar_lm.train(train_sentences)

  0%|          | 0/174519 [00:00<?, ?it/s]

In [20]:
def correct_text_with_grammar(text, grammar_model, top_k=5):
    tokens = [token.text for token in tokenize(text)]
    
    corrections = [] 
    original_structure = []
    
    for token in tokens:
        word = token.lower()
        
        is_upper = token[0].isupper() if token else False
        is_punct = word in punct or not word.strip()
        original_structure.append({'is_upper': is_upper, 'is_punct': is_punct, 'orig': token})
        
        if is_punct:
            corrections.append([word])
            continue
            
        if word in vocab:
            corrections.append([word])
        else:
            cands = list(candidates(word))
            
            cands = sorted(cands, key=P, reverse=True)[:top_k]
            
            if not cands:
                corrections.append([word])
            else:
                corrections.append(cands)

    possible_sentences_tokens = list(itertools.product(*corrections))
    
    def get_score(tokens_tuple):
        sent_str = " ".join(tokens_tuple)
        return grammar_model.compute_sentence_probability(sent_str, use_log=True)

    best_tokens = max(possible_sentences_tokens, key=get_score)
    
    final_tokens = []
    for i, word in enumerate(best_tokens):
        if original_structure[i]['is_upper']:
            final_tokens.append(word.capitalize())
        else:
            final_tokens.append(word)
            
    return " ".join(final_tokens)

In [21]:
def correct_text_simple(text):
    tokens = [t.text for t in tokenize(text)]
    
    corrected_tokens = []
    for token in tokens:
        if token.lower() in punct or not token.strip():
            corrected_tokens.append(token)
            continue
        
        corrected_word = correction(token.lower())
        
        if token and token[0].isupper():
            corrected_word = corrected_word.capitalize()
        
        corrected_tokens.append(corrected_word)
    
    result_parts = []
    for i, token in enumerate(corrected_tokens):
        if i > 0 and token not in punct and corrected_tokens[i-1] not in punct:
            result_parts.append(' ')
        result_parts.append(token)
    
    return ''.join(result_parts)

In [22]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

def evaluate_grammar_model(grammar_model):
    y_true_all = []
    y_actual_all = []
    y_pred_grammar = []
    
    for i in tqdm(range(len(true)), desc="Оценка модели"):
        bad_sent = bad[i]
        true_sent = true[i]
        
        word_pairs = align_words(true_sent, bad_sent)
        
        corrected_sent_str = correct_text_with_grammar(bad_sent, grammar_model)
        
        corrected_tokens = corrected_sent_str.lower().split()
        corrected_tokens = [t.strip(punctuation) for t in corrected_tokens]
        corrected_tokens = [t for t in corrected_tokens if t]
        
        if len(corrected_tokens) != len(word_pairs):
             corrected_tokens = [pair[1] for pair in word_pairs]

        for j, (true_word, actual_word) in enumerate(word_pairs):
            y_true_all.append(true_word)
            y_actual_all.append(actual_word)
            y_pred_grammar.append(corrected_tokens[j])

    metrics = calculate_metrics(y_true_all, y_actual_all, y_pred_grammar)
    return metrics

In [23]:
metrics_result = evaluate_grammar_model(grammar_lm)

pprint(metrics_result)

Оценка модели:   0%|          | 0/915 [00:00<?, ?it/s]

{'broken_correct_words': 0.07315952681750316,
 'fixed_mistakes': 0.5,
 'total_accuracy': 0.8718359179589795}


In [24]:
def extract_clean_words(sentence_str):
    tokens = sentence_str.lower().split() 
    
    clean_words = [token.strip(punctuation) for token in tokens]
    clean_words = [word for word in clean_words if word]
    
    return clean_words

def evaluate_all_data(grammar_model=None, alpha=0.3):
    y_true_all = []
    y_actual_all = []
    y_pred_simple = []
    y_pred_grammar = []
    
    
    for i in tqdm(range(len(true)), desc="Обработка пар предложений"):
        true_sent = true[i]
        bad_sent = bad[i]
        
        word_pairs = align_words(true_sent, bad_sent)
        
        simple_words = []
        for correct_word, wrong_word in word_pairs:
            corrected = correction(wrong_word) 
            simple_words.append(corrected)
        
        grammar_corrected_sent = correct_text_with_grammar(bad_sent, grammar_model)
        
        grammar_words = extract_clean_words(grammar_corrected_sent)
        
        final_grammar_words = []
        if len(grammar_words) != len(word_pairs):
            final_grammar_words = simple_words
        else:
            final_grammar_words = grammar_words
        
        for j, (correct_word, wrong_word) in enumerate(word_pairs):
            y_true_all.append(correct_word)
            y_actual_all.append(wrong_word)
            
            y_pred_simple.append(simple_words[j])
            y_pred_grammar.append(final_grammar_words[j])
            
    metrics_simple = calculate_metrics(y_true_all, y_actual_all, y_pred_simple)
    metrics_grammar = calculate_metrics(y_true_all, y_actual_all, y_pred_grammar)

    return {
        'metrics_simple': metrics_simple,
        'metrics_grammar': metrics_grammar,
        'y_true': y_true_all,
        'y_actual': y_actual_all,
        'y_pred_simple': y_pred_simple,
        'y_pred_grammar': y_pred_grammar
    }

In [27]:
results = evaluate_all_data(grammar_model=grammar_lm)

# --- Вывод сравнительных метрик ---
metrics_simple = results['metrics_simple']
metrics_grammar = results['metrics_grammar']

print("анализ метрик моделей")

simple_acc = metrics_simple['total_accuracy']
grammar_acc = metrics_grammar['total_accuracy']
acc_diff = grammar_acc - simple_acc
print(f"Общая точность      | {simple_acc:.4f}       | {grammar_acc:.4f}           (Разница: {acc_diff:+.4f})")

simple_fixed = metrics_simple['fixed_mistakes']
grammar_fixed = metrics_grammar['fixed_mistakes']
fixed_diff = grammar_fixed - simple_fixed
print(f"Исправлено ошибок   | {simple_fixed:.4f}       | {grammar_fixed:.4f}           (Разница: {fixed_diff:+.4f})")

simple_broken = metrics_simple['broken_correct_words']
grammar_broken = metrics_grammar['broken_correct_words']
broken_diff = grammar_broken - simple_broken
print(f"Сломано правильных  | {simple_broken:.4f}       | {grammar_broken:.4f}           (Разница: {broken_diff:+.4f})")


#анализ примеров, где грамматическая модель лучше справилась, чем обычная
y_true = results['y_true']
y_actual = results['y_actual']
y_pred_simple = results['y_pred_simple']
y_pred_grammar = results['y_pred_grammar']
total_words = len(y_true)

helpful_examples = []
harmful_examples = []
differences_found = 0

for i in range(total_words):
    true_w = y_true[i]
    actual_w = y_actual[i]
    pred_simple = y_pred_simple[i]
    pred_grammar = y_pred_grammar[i]
    
    if pred_simple != pred_grammar:
        differences_found += 1
        
        simple_correct = (true_w == pred_simple)
        grammar_correct = (true_w == pred_grammar)
        
        if grammar_correct and not simple_correct:
            helpful_examples.append({'Ист': true_w, 'Ошб': actual_w, 'Простой': pred_simple, 'Грамматика': pred_grammar})
            
        elif simple_correct and not grammar_correct:
             harmful_examples.append({'Ист': true_w, 'Ошб': actual_w, 'Простой': pred_simple, 'Грамматика': pred_grammar})


print(f"расхождений всего: {differences_found}")

# Функция для вывода примера
def print_example(ex, outcome):
    print(f"  Исходное слово с ошибкой: '{ex['Ошб']}' (правильное слово: {ex['Ист']})")
    print(f"    Простой корректор (P(w)):    '{ex['Простой']}'")
    print(f"    Грамматический корректор: '{ex['Грамматика']}' ({outcome})")
    print("---")


print("\nСлучаи, когда грамматическая модель улучшила результат (исправила ошибку, которую пропустила обычная модель):")
if helpful_examples:
    for i, ex in enumerate(helpful_examples[:5]):
        print(f"Пример {i+1}:")
        print_example(ex, "ВЕРНО")
else:
     print("Явных примеров улучшения не обнаружено")


print("\nСлучаи, когда грамматическая модель ухудшила результат:")
if harmful_examples:
     for i, ex in enumerate(harmful_examples[:5]):
        print(f"Пример {i+1}:")
        print_example(ex, "НЕВЕРНО")
else:
    print("Явных примеров ухудшения не обнаружено")

Обработка пар предложений:   0%|          | 0/915 [00:00<?, ?it/s]

анализ метрик моделей
Общая точность      | 0.8708       | 0.8709           (Разница: +0.0001)
Исправлено ошибок   | 0.5116       | 0.5124           (Разница: +0.0008)
Сломано правильных  | 0.0760       | 0.0760           (Разница: +0.0000)
расхождений всего: 449

Случаи, когда грамматическая модель улучшила результат (исправила ошибку, которую пропустила обычная модель):
Пример 1:
  Исходное слово с ошибкой: 'основая' (правильное слово: основная)
    Простой корректор (P(w)):    'основан'
    Грамматический корректор: 'основная' (ВЕРНО)
---
Пример 2:
  Исходное слово с ошибкой: 'сранно' (правильное слово: странно)
    Простой корректор (P(w)):    'санно'
    Грамматический корректор: 'странно' (ВЕРНО)
---
Пример 3:
  Исходное слово с ошибкой: 'нмного' (правильное слово: немного)
    Простой корректор (P(w)):    'много'
    Грамматический корректор: 'немного' (ВЕРНО)
---
Пример 4:
  Исходное слово с ошибкой: 'самыи' (правильное слово: самый)
    Простой корректор (P(w)):    'самым'
   

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (обратите внимание, что для одного удаления может быть несколько правильных слов!) 
3) При исправлении слова с опечаткой сначала само слово проверятся по словарю удаления, а затем для этого слова генерируются все варианты удалений, и каждый вариант проверяется по словарю удалений. Если в словаре удалений таким образом находится совпадение, то соответствующее ему правильное слово становится исправлением.
Если совпадений несколько, то выбирается наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [28]:
def generate_deletes(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return set(deletes)

symspell_dict = {}

for correct_word in tqdm(vocab.keys()):
    if correct_word not in symspell_dict:
        symspell_dict[correct_word] = set()
    symspell_dict[correct_word].add(correct_word)
    
    deletes = generate_deletes(correct_word)
    for deleted_word in deletes:
        if deleted_word not in symspell_dict:
            symspell_dict[deleted_word] = set()
        symspell_dict[deleted_word].add(correct_word)

def symspell_correction(word):
    if word in vocab:
        return word
        
    candidates = set()
    
    if word in symspell_dict:
        candidates.update(symspell_dict[word])
        
    word_deletes = generate_deletes(word)
    for d in word_deletes:
        if d in symspell_dict:
            candidates.update(symspell_dict[d])
            
    if not candidates:
        return word
        
    return max(candidates, key=P)

print(f"сонце -> {symspell_correction('сонце')}") # удаление
print(f"солнцее -> {symspell_correction('солнцее')}") # вставка

  0%|          | 0/368802 [00:00<?, ?it/s]

сонце -> конце
солнцее -> солнце


In [29]:
y_true_sym = []
y_actual_sym = []
y_pred_sym = []

for i in tqdm(range(len(true)), desc="SymSpell Evaluation"):
    true_sent = true[i]
    bad_sent = bad[i]
    
    word_pairs = align_words(true_sent, bad_sent)
    
    bad_tokens = bad_sent.lower().split()
    bad_tokens = [t.strip(punctuation) for t in bad_tokens]
    bad_tokens = [t for t in bad_tokens if t]
    
    if len(bad_tokens) != len(word_pairs):
        bad_tokens = [pair[1] for pair in word_pairs]

    for j, (correct_w, wrong_w) in enumerate(word_pairs):
        token_to_fix = bad_tokens[j]
        predicted = symspell_correction(token_to_fix)
        
        y_true_sym.append(correct_w)
        y_actual_sym.append(wrong_w)
        y_pred_sym.append(predicted)

metrics_symspell = calculate_metrics(y_true_sym, y_actual_sym, y_pred_sym)

print(f"Общая точность      : {metrics_symspell['total_accuracy']:.2%}")
print(f"Исправлено ошибок   : {metrics_symspell['fixed_mistakes']:.2%}")
print(f"Сломано правильных  : {metrics_symspell['broken_correct_words']:.2%}")

SymSpell Evaluation:   0%|          | 0/915 [00:00<?, ?it/s]

Общая точность      : 87.85%
Исправлено ошибок   : 40.92%
Сломано правильных  : 5.20%


# Задание 3 (2 балла)

Используя любой из алгоритмов из семинара или домашки, детально проанализируйте получаемые ошибки. Улучшите алгоритм так, чтобы исправить ошибки. Улучшения в алгоритме должны быть общими, не привязанными к конкретным словам (например, словарь исключений не будет считаться). За каждое улучшение, которое исправляет 5+ ошибок вы получите 0.5 балла (максимум 2 в целом)